In [1]:
import numpy as np
from scipy.optimize import fsolve

# GMSH commands definition

In [2]:
## GMSH commands
def point (*args):
    global last_point_id
    if len(args)==4:
        point_id=last_point_id+1
        last_point_id=point_id
    else:
        point_id=args[4]
        last_point_id=max(point_id,last_point_id)
    with open(file_name, 'a') as file:
        print(f"Point({point_id}) = {{{args[0]}, {args[1]}, {args[2]}, {args[3]}}};", file=file)

def point_rot(*args):
    #args=[x,y,z,h,[xc, yc],alpha_rot,point_id]
    alpha=args[5]
    x_rot=(args[0]-args[4][0])*np.cos(alpha)-(args[1]-args[4][1])*np.sin(alpha)+args[4][0]
    y_rot=(args[0]-args[4][0])*np.sin(alpha)+(args[1]-args[4][1])*np.cos(alpha)+args[4][1]
    if len(args)==6:
        point(x_rot,y_rot,args[2],args[3])
    else:
        point(x_rot,y_rot,args[2],args[3],args[6])

def line (*args):
    global last_line_id
    if len(args)==2:
        line_id=last_line_id+1
        last_line_id=line_id
    else:
        line_id=args[2]
        last_line_id=max(line_id,last_line_id)
    with open(file_name, 'a') as file:
        print(f"Line({line_id}) = {{{args[0]}, {args[1]}}};", file=file)


def transline(*args):
    #Should be used actually called the line_id, since line must already exist
    global last_transline_id
    if len(args)==2:
        transline_id=last_transline_id+1
    else:
        transline_id=args[2]
    with open(file_name, 'a') as file:
        print(f"Transfinite Line({transline_id}) = {args[0]} Using Progression {args[1]};", file=file)
        last_transline_id=last_transline_id+1

def circle (*args):
    global last_line_id
    if len(args)==3:
        line_id=last_line_id+1
        last_line_id=line_id
    else:
        line_id=args[3]
        last_line_id=max(line_id,last_line_id)
    with open(file_name, 'a') as file:
        print(f"Circle({line_id}) = {{{args[0]}, {args[1]}, {args[2]}}};", file=file)
        last_line_id=last_line_id+1

def loop(vec,num):
    global last_line_loop
    if num==0:
        loop_id=last_line_loop+1
        last_line_loop=loop_id
    else:
        loop_id=num
        last_line_loop=max(loop_id,last_line_loop)
    with open(file_name, 'a') as file:
        print(f"Line Loop({loop_id}) = {{{', '.join(map(str, vec))}}};",file=file)

def surface(num, str):
    global last_surface_id
    if num==0:
        num_in=last_surface_id+1
        last_surface_id=num_in
    else:
        num_in=num
        last_surface_id=max(num,last_surface_id)
    with open(file_name, 'a') as file:
        print(f"Plane Surface({num_in}) = {{{str}}};",file=file)

def transurface(vec,num):
    global last_transurface
    if num==0:
        transurface_id=last_transurface+1
    else:
        transurface_id=num
    with open(file_name, 'a') as file:
        print(f"Transfinite Surface({transurface_id}) = {{{', '.join(map(str, vec))}}};",file=file)
        print(f"Recombine Surface({transurface_id});",file=file)
        last_transurface=last_transurface+1

def spline(*args):
    global last_line_id
    if len(args)==1:
        line_id=last_line_id+1
        last_line_id=line_id
    else:
        line_id=args[2]
        last_line_id=max(line_id,last_line_id)
    with open(file_name, 'a') as file:
        print(f"Spline({line_id}) = {{{', '.join(map(str, args[0]))}}};",file=file)

def phys_surf(surf_id):
    with open(file_name, 'a') as file:
        print(f"Physical Surface({surf_id}) = {{{surf_id}}};",file=file)

def import_profile(point_vec,center,alpha,surface_id):
    alpha=-alpha*np.pi/180
    startpoint=last_point_id+1
    startline=last_line_id
    point_rot=np.zeros((2*len(point_vec),2))
    for i in np.arange(0,len(point_vec),1):
        x_temp=point_vec[i][0]
        y_temp=point_vec[i][1]
        point_rot[i][0]=x_temp*np.cos(alpha)-y_temp*np.sin(alpha)+center[0]
        point_rot[i][1]=x_temp*np.sin(alpha)+y_temp*np.cos(alpha)+center[1]
        point(point_rot[i][0],point_rot[i][1],0,h)
    for i in np.arange(0,len(point_vec)-1,1):
        line(startpoint+i,startpoint+i+1)
    line(startpoint+len(point_vec)-1,startpoint)
    loop(np.arange(startline+1,last_line_id+1,1),surface_id)
    global field_id
    field_id=field_id+1
    line_string = "{" + ', '.join(map(str, np.arange(startline + 1, last_line_id + 1, 1))) + "};"
    boundary_layer_field=[f"Field[{field_id}]=BoundaryLayer;",
                          f"Field[{field_id}].CurvesList={line_string}",
                          f"Field[{field_id}].Quads=1;",
                          f"Field[{field_id}].Ratio=1.1;",
                          f"Field[{field_id}].Size=0.00001;",
                          f"Field[{field_id}].Thickness=0.007;",
                          f"Field[{field_id}].FanPointsList={{{int(startpoint+len(point_vec)/2)}}};",
                          f"Field[{field_id}].FanPointsSizesList={{{40}}};",
                          f"BoundaryLayer Field = {field_id};\n"]
    write_custom(boundary_layer_field)

def build_naca_point_vec(naca_id, c, flag, num_points=50):
    m=int(naca_id[0])/100
    p=int(naca_id[1])/10
    t=int(naca_id[2:])/100
    x=c*(1 - np.cos(np.linspace(0,np.pi,num_points)))/2
    if flag:
        #flag=1 -> bordo d'uscita aguzzo, flag=0 -> profilo naca stanard, problemi numerici
        #Lo strato limite su import_profile funziona solo se flag=1
        last_num=-0.1036
    else:
        last_num=-0.1015
    yt=5*t*c*(0.2969*np.sqrt(x/c)+(-0.1260)*(x/c)+(-0.3516)*(x/c)**2+0.2843*(x/c)**3+(last_num)*(x/c)**4)
    point_vec=np.zeros((2*num_points,2))
    if p==0 and m==0:
        for i in np.arange(0,num_points,1):
            point_vec[i]=[x[i],yt[i]]
            point_vec[2*num_points-1-i]=[x[i],-yt[i]]
    else:
        i=0
        yc=np.zeros((num_points,1))
        dycdx=np.zeros((num_points,1))
        while 1:
            if x[i]>p*c:
                break
            yc[i]=x[i]*m*(2*p-(x[i]/c))/(p**2)
            dycdx[i]=2*m*(p-(x[i]/c))/(p**2)
            i=i+1
        while i<num_points:
            yc[i]=m*(c-x[i])*(1+(x[i]/c)-2*p)/((1-p)**2)
            dycdx[i]=2*m*(p-(x[i]/c))/((1-p)**2)
            i=i+1
        theta=np.arctan(dycdx)
        for i in np.arange(0,num_points,1):
            point_vec[i]=[float(arr[0]) for arr in [x[i]-yt[i]*np.sin(theta[i]),yc[i]+yt[i]*np.cos(theta[i])]]
            point_vec[2*num_points-1-i]=[float(arr[0]) for arr in [x[i]+yt[i]*np.sin(theta[i]),yc[i]-yt[i]*np.cos(theta[i])]]
    if flag:
        point_vec=np.delete(point_vec,num_points,axis=0)
        point_vec=np.delete(point_vec,2*num_points-2,axis=0)
    return point_vec

def write_custom(lines_to_add):
    with open(file_name, 'a') as file:
        file.write('\n'.join(lines_to_add))

# Geometrical parameters of the mesh

In [3]:
#-------------------------PARAMETERS SETTINGS-----------------------------------
# Flow definition
rho = 1.185
mu = 1.785e-5
Uinf = 1.8076
yp = 1.0
# Geometry definition
# inlet diameter
d1=0.2
# outlet diameter
d2=0.2
# lenght of inlet
l1=5*d1
# lenght of outlet
l2=10*d1

# fraction of the inlet inside elbow block
t1=0.2*l1
# fraction of the inlet inside elbow block
t2=0.3*l2
# Radius of curvature n.d
Rc_in = 1.58
Rc_ext = 1.58
radius_in=(Rc_in-0.5)*d1  # inner radius of curvature
radius_ex=(Rc_ext+0.5)*d1  # outter radius of curvature

# Mesh parameters
h=0.5       # standard element size
# inlet
dxc1=0.03*d1  # x-lenth for elements in the center
dyc1=0.02*d1  # y-lenth for elements in the center
# outlet
dxc2=0.02*d1  # x-lenth for elements in the center
dyc2=0.03*d1  # y-lenth for elements in the center
# elbow
delb=0.015*d1 # standar elbow element size
delb_precise=0.008*d1 # refined element size in elbow

# y plus
def deltascalc(Uinf,rho,mu,L,yp):
    Rex = rho*Uinf*L/mu
    cf = 0.079/Rex**(0.25) # Carloni et al. 2022
    tw = cf*rho*Uinf**2/2
    up = (tw/rho)**0.5
    deltas = yp*mu/(up*rho)
    return deltas

deltas = deltascalc(Uinf,rho,mu,d1,yp)
x_0=deltas  # first element position near wall
# calculations for interface matching
x_N=dyc1                # y-size in inlet/outlet
x_N2=delb_precise/1.1   # y-size in refined section of elbow
s1=0.02                 # BL thickness in inlet
s2=s1                   # BL thickness in outlet



# Elbow mesh generation

In [4]:
# elbow definition
alpha_rot=0             # elbow degree (pi/2-alpha_rot)
sharp_check=1e-8
################################################################################

nxc1=int((l1-t1)/dxc1)      # Number of x elements in inlet
nyc1=int((d1-s1-s2)/dyc1)   # Number of y elements in inlet
nxc2=int((d2-s1-s2)/dxc2)   # Number of y elements in outlet
nyc2=int((l2-t2)/dyc2)      # Number of y elements in outlet
nblockin=int(2*s2/delb_precise)+1
ncurvext=int(radius_ex*(np.pi*0.5-alpha_rot*np.pi/180)/delb_precise)+1

def equations(vars):
    x, y = vars
    eq1 = x**y - x_N/x_0
    eq2 = (1-x**(y+1))/(1-x) - s1/x_0
    return [eq1, eq2]

initial_guess = [1.1, 50]
solution = fsolve(equations, initial_guess)
nsl1=int(solution[1])+3
nsl2=nsl1
prog=1/solution[0]
# Modifiy the file name here
file_name='mesh.geo'
last_point_id=0
last_line_id=0
last_transline_id=0
last_line_loop=0
last_surface_id=0
last_transurface=0
last_phys_surf=0
field_id=0


with open(file_name, 'w') as file:
    print(f"//MESH FILE\n", file=file)

if ncurvext<3:
    ncurvext=3

alpha_rot=alpha_rot*np.pi/180
l3_temp=np.sqrt(d1**2+d2**2-2*d1*d2*np.cos(np.pi/2-alpha_rot))
alpha3_temp=np.arccos((-d1**2+d2**2+l3_temp**2)/(2*d2*l3_temp))
l4_temp=l3_temp*np.sin(np.pi/2-alpha3_temp)/np.sin(np.pi/2+alpha_rot)

Sum_geom=(1-(1/prog)**(nsl1-1))/(1-(1/prog))
x_0_real=s1/Sum_geom
x_N_temp=0
i=0
while x_N_temp<x_N2:
    i=i+1
    x_N_temp=x_0_real*((1/prog)**i)
nsl_2=i+1
#nsl_2=np.emath.logn((1/prog),1-(thick2/x_0_real)*(1-1/prog))+1
thick2=x_0_real*(1-(1/prog)**(nsl_2-1))/(1-(1/prog))
thick1=thick2

lcorr_in=radius_in/np.tan((np.pi/2+alpha_rot)/2)
lcorr_ex=radius_ex/np.tan((np.pi/2+alpha_rot)/2)
# Point definition

# Inlet points
point(0,0,0,h,1)
point(l1-t1-lcorr_in,0,0,h,2)
point(l1-t1-lcorr_in,s1,0,h,3)
point(0,s1,0,h,4)
point(0,d1-s2,0,h,5)
point(l1-t1-lcorr_in,d1-s2,0,h,6)
point(l1-t1-lcorr_in,d1,0,h,7)
point(0,d1,0,h,8)
# Outlet points
cen=[l1,0]
point_rot(l1,-t2-lcorr_in,0,h,cen,alpha_rot,9)
point_rot(l1,-l2,0,h,cen,alpha_rot,10)
point_rot(l1+s1,-l2,0,h,cen,alpha_rot,11)
point_rot(l1+s1,-t2-lcorr_in,0,h,cen,alpha_rot,12)
point_rot(l1+d2-s2,-t2-lcorr_in,0,h,cen,alpha_rot,13)
point_rot(l1+d2-s2,-l2,0,h,cen,alpha_rot,14)
point_rot(l1+d2,-l2,0,h,cen,alpha_rot,15)
point_rot(l1+d2,-t2-lcorr_in,0,h,cen,alpha_rot,16)

# Elbow points
point(l1-lcorr_in,0,0,h,17)                       #first point circle
if radius_in>=sharp_check:
    point_rot(l1,-lcorr_in,0,h,cen,alpha_rot,18)  # last point circle
    point(l1-lcorr_in,-radius_in,0,h,29)          # centre of circle
    #point(l1,0,0,h,19)
point(l1+l4_temp-lcorr_ex-2*s2,d1,0,h,20)
point(l1+l4_temp-lcorr_ex,d1,0,h,21)
if radius_ex>=sharp_check:
    point(l1+l4_temp+lcorr_ex*np.sin(alpha_rot),d1-lcorr_ex*np.cos(alpha_rot),0,h,22)
    point(l1+l4_temp-lcorr_ex,d1-radius_ex,0,h,30)
point(l1+l4_temp+(lcorr_ex+2*s2)*np.sin(alpha_rot),d1-(lcorr_ex+2*s2)*np.cos(alpha_rot),0,h,23)
point(l1+l4_temp+(lcorr_ex+2*s2)*np.sin(alpha_rot)-thick2*np.cos(alpha_rot),d1-(lcorr_ex+2*s2)*np.cos(alpha_rot)-thick2*np.sin(alpha_rot),0,h,24)
if radius_ex>1.1*s2:
    point(l1+l4_temp+lcorr_ex*np.sin(alpha_rot)-thick2*np.cos(alpha_rot),d1-lcorr_ex*np.cos(alpha_rot)-thick2*np.sin(alpha_rot),0,h,25)
    point(l1+l4_temp-lcorr_ex,d1-thick2,0,h,26)
else:
    if radius_ex>=sharp_check:
        point(l1+l4_temp+lcorr_ex*np.sin(alpha_rot)-thick2*np.cos(alpha_rot)/(1+np.sin(alpha_rot)),d1-lcorr_ex*np.cos(alpha_rot)-(thick2+thick2*np.sin(alpha_rot))/(1+np.sin(alpha_rot)),0,h,25)
        point(l1+l4_temp-lcorr_ex-thick2*np.cos(alpha_rot)/(1+np.sin(alpha_rot)),d1-radius_ex-(thick2+thick2*np.sin(alpha_rot))/(1+np.sin(alpha_rot)),0,h,31)
    point(l1+l4_temp-lcorr_ex-thick2*np.cos(alpha_rot)/(1+np.sin(alpha_rot)),d1-(thick2+thick2*np.sin(alpha_rot))/(1+np.sin(alpha_rot)),0,h,26)

point(l1+l4_temp-lcorr_ex-2*s2,d1-thick2,0,h,27)
#point(l1+l4_temp,d1,0,h,28)


# Line definition
line(1,2,1)
line(2,3,2)
line(3,4,3)
line(4,1,4)
line(3,6,5)
line(4,5,6)
line(5,6,7)
line(6,7,8)
line(7,8,9)
line(8,5,10)
line(9,10,11)
line(10,11,12)
line(11,12,13)
line(12,9,14)
line(12,13,15)
line(11,14,16)
line(13,14,17)
line(14,15,18)
line(15,16,19)
line(16,13,20)
if radius_in>=sharp_check:
    line(2,17,21)
    circle(17,29,18,22)
    line(18,9,23)
if radius_ex>=sharp_check:
    line(20,21,25)
    circle(21,30,22,26)
    line(22,23,27)
    line(24,25,29)
    line(26,27,31)
    if radius_ex>1.1*s2:
        circle(26,30,25,30)
    else:
        circle(26,31,25,30)
line(7,20,24)
line(23,24,28)
line(27,20,32)
line(23,16,33)
if radius_ex<sharp_check:
    line(20,21,39)
    line(21,23,34)
    line(24,26,35)
    line(26,27,36)
if radius_in<sharp_check:
    line(2,17,37)
    line(17,9,38)

#----------------Mesh definition algorithm--------------------------------------
transline(nxc1, 1, 1)
transline(nsl1, 1/prog, 2)
transline(nxc1, 1, 3)
transline(nsl1, prog, 4)
transline(nyc1, 1, 5)
transline(nyc1, 1, 6)
transline(nxc1, 1, 7)
transline(nsl2, prog, 8)
transline(nxc1, 1, 9)
transline(nsl2, 1/prog, 10)

transline(nyc2, 1, 11)
transline(nsl1, 1/prog, 12)
transline(nyc2, 1, 13)
transline(nsl1, prog, 14)
transline(nxc2, 1, 15)
transline(nxc2, 1, 16)
transline(nyc2, 1, 17)
transline(nsl2, prog, 18)
transline(nyc2, 1, 19)
transline(nsl2, 1/prog, 20)

transline(nblockin, 1, 25)
transline(ncurvext, 1, 26)
transline(nblockin, 1, 27)
transline(nsl_2, 1/prog, 28)
transline(nblockin, 1, 29)
transline(ncurvext, 1, 30)
transline(nblockin, 1, 31)
transline(nsl_2, prog, 32)
transline(nblockin, 1, 34)
transline(nblockin, 1, 35)
transline(nblockin, 1, 36)
transline(nblockin, 1, 39)

if radius_in>=sharp_check and radius_ex>=sharp_check:
    loop((25,26,27,28,29,-30,31,32),1)
    loop((2,5,8,24,-32,-31,30,-29,-28,33,20,-15,14,-23,-22,-21),2)
if radius_in<sharp_check and radius_ex>=sharp_check:
    loop((25,26,27,28,29,-30,31,32),1)
    loop((2,5,8,24,-32,-31,30,-29,-28,33,20,-15,14,-38,-37),2)
if radius_in>=sharp_check and radius_ex<sharp_check:
    loop((39,34,28,35,36,32),1)
    loop((2,5,8,24,-32,-36,-35,-28,33,20,-15,14,-23,-22,-21),2)
if radius_in<sharp_check and radius_ex<sharp_check:
    loop((39,34,28,35,36,32),1)
    loop((2,5,8,24,-32,-36,-35,-28,33,20,-15,14,-38,-37),2)

loop((1,2,3,4),3)
loop((5,-7,-6,-3),4)
loop((7,8,9,10),5)
loop((11,12,13,14),6)
loop((16,-17,-15,-13),7)
loop((17,18,19,20),8)

surface(1,"1")
surface(2,"2")
surface(3,"3")
surface(4,"4")
surface(5,"5")
surface(6,"6")
surface(7,"7")
surface(8,"8")

transurface((20,23,24,27),1)
transurface((1,2,3,4),3)
transurface((3,4,5,6),4)
transurface((5,6,7,8),5)
transurface((9,10,11,12),6)
transurface((11,12,13,14),7)
transurface((13,14,15,16),8)

profile_imp=build_naca_point_vec("0015",d1/4,1)
#import_profile(profile_imp,[l1+d1/4,0],90,10)
profile_imp2=build_naca_point_vec("2415",d1/3,1)
#import_profile(profile_imp2,[l1+d1/2,d1/2],60,11)
#profile_imp3=build_naca_point_vec("6412",d/1.5,1)
#import_profile(profile_imp3,[L/2-3*d,d/2],15,0)
#surface(10,"2,10,11")

Thickness=thick1*1.03
Ratio=1/prog
Sum_geom=(1-(1/prog)**(nsl1-1))/(1-(1/prog))
Size=s1/Sum_geom
field_id=field_id+1
boundary_layer_field_sharp=[f"Field[{field_id}]=BoundaryLayer;",
                            f"Field[{field_id}].CurvesList={{37, 38}};",
                            f"Field[{field_id}].Quads=1;",
                            f"Field[{field_id}].Ratio={Ratio};",
                            f"Field[{field_id}].Size={Size};",
                            f"Field[{field_id}].Thickness={Thickness};",
                            f"Field[{field_id}].FanPointsList={{17}};",
                            f"Field[{field_id}].FanPointsSizesList={{{20}}};",
                            f"BoundaryLayer Field = {field_id};\n"]
boundary_layer_field_round=[f"Field[{field_id}]=BoundaryLayer;",
                            f"Field[{field_id}].CurvesList={{21,22,23}};",
                            f"Field[{field_id}].Quads=1;",
                            f"Field[{field_id}].Ratio={Ratio};",
                            f"Field[{field_id}].Size={Size};",
                            f"Field[{field_id}].Thickness={Thickness};",
                            f"BoundaryLayer Field = {field_id};\n"]
if radius_in>=sharp_check:
    write_custom(boundary_layer_field_round)
else:
    write_custom(boundary_layer_field_sharp)

Thickness=thick2*1.03
Ratio=1/prog
Sum_geom=(1-(1/prog)**(nsl2-1))/(1-(1/prog))
Size=s2/Sum_geom
field_id=field_id+1
boundary_layer_field=[f"Field[{field_id}]=BoundaryLayer;",
                      f"Field[{field_id}].CurvesList={{24}};",
                      f"Field[{field_id}].Quads=1;",
                      f"Field[{field_id}].Ratio={Ratio};",
                      f"Field[{field_id}].Size={Size};",
                      f"Field[{field_id}].Thickness={Thickness};",
                      f"BoundaryLayer Field = {field_id};\n"]
write_custom(boundary_layer_field)
Thickness=thick2*1.03
Ratio=1/prog
Sum_geom=(1-(1/prog)**(nsl2-1))/(1-(1/prog))
Size=s1/Sum_geom
field_id=field_id+1
boundary_layer_field=[f"Field[{field_id}]=BoundaryLayer;",
                      f"Field[{field_id}].CurvesList={{33}};",
                      f"Field[{field_id}].Quads=1;",
                      f"Field[{field_id}].Ratio={Ratio};",
                      f"Field[{field_id}].Size={Size};",
                      f"Field[{field_id}].Thickness={Thickness};",
                      f"BoundaryLayer Field = {field_id};\n"]
write_custom(boundary_layer_field)

field_id=field_id+1
distance_field=[f"Field[{field_id}] = Distance;",
                f"Field[{field_id}].NodesList = {18};",
                f"Field[{field_id+1}] = Threshold;",
                f"Field[{field_id+1}].IField = {field_id};",
                f"Field[{field_id+1}].LcMin = {delb};",
                f"Field[{field_id+1}].LcMax = {delb};",
                f"Field[{field_id+1}].DistMin = 0.002;",
                f"Field[{field_id+1}].DistMax = 0.002;",
                f"//Background Field = {field_id+1};\n"]
#write_custom(distance_field)

field_id=field_id+2
box_field=[f"Field[{field_id}]=Box;",
           f"Field[{field_id}].Thickness={d1*0.1};",
           f"Field[{field_id}].VIn={delb_precise};",
           f"Field[{field_id}].VOut={delb};",
           f"Field[{field_id}].XMax={l1+d2};",
           f"Field[{field_id}].XMin={l1-lcorr_in};",
           f"Field[{field_id}].YMax={s2};",
           f"Field[{field_id}].YMin={-t2/1.5};",
           f"Background Field = {field_id};\n"]
write_custom(box_field)

lines_to_add_rr = [
    'Physical Surface("VOLUME") = {1,2,3,4,5,6,7,8};',
    'Physical Line("wall") = {1,21,22,23,11,9,24,25,26,27,33,19};',
    'Physical Line("inlet") = {4,6,10};',
    'Physical Line("outlet") = {12,16,18};\n'
]
lines_to_add_sr = [
    'Physical Surface("VOLUME") = {1,2,3,4,5,6,7,8};',
    'Physical Line("wall") = {1,37,38,11,9,24,25,26,27,33,19};',
    'Physical Line("inlet") = {4,6,10};',
    'Physical Line("outlet") = {12,16,18};\n'
]
lines_to_add_rs = [
    'Physical Surface("VOLUME") = {1,2,3,4,5,6,7,8};',
    'Physical Line("wall") = {1,21,22,23,11,9,24,39,34,33,19};',
    'Physical Line("inlet") = {4,6,10};',
    'Physical Line("outlet") = {12,16,18};\n'
]
lines_to_add_ss = [
    'Physical Surface("VOLUME") = {1,2,3,4,5,6,7,8};',
    'Physical Line("wall") = {1,37,38,11,9,24,39,34,33,19};',
    'Physical Line("inlet") = {4,6,10};',
    'Physical Line("outlet") = {12,16,18};\n'
]

line_string = "{" + ', '.join(map(str, np.arange(1, last_line_loop+1, 1))) + "};"
lines_to_add2 = [
    f"Plane Surface(1)={line_string}"
]
#raise SystemExit

if radius_in>=sharp_check and radius_ex>=sharp_check:
    write_custom(lines_to_add_rr)
if radius_in<sharp_check and radius_ex>=sharp_check:
    write_custom(lines_to_add_sr)
if radius_in>=sharp_check and radius_ex<sharp_check:
    write_custom(lines_to_add_rs)
if radius_in<sharp_check and radius_ex<sharp_check:
    write_custom(lines_to_add_ss)
# Mesh generation and name
D = str(d1)
D = D.replace('.','_')
re = str(round(radius_ex,3))
re = re.replace('.','_')
ri = str(round(radius_in,3))
ri = ri.replace('.','_')
li = str(l1)
li = li.replace('.','_')
lo = str(l2)
lo = lo.replace('.','_')
mesh_suffix = 'D'+D+'_re'+re+'_ri'+ri+'_li'+li+'_lo'+lo
mesh_name = '"mesh'+mesh_suffix+'.su2"'
meshing = [
    'Mesh 2;',
    'Save '+mesh_name+';\n'
]
write_custom(meshing)

# Channel mesh

In [5]:
# elbow definition
alpha_rot=0             # elbow degree (pi/2-alpha_rot)
sharp_check=1e-8
lc = 100*d1
################################################################################

nxc1=int((l1-t1)/dxc1)      # Number of x elements in inlet
nyc1=int((d1-s1-s2)/dyc1)   # Number of y elements in inlet
nxc2=int((d2-s1-s2)/dxc2)   # Number of y elements in outlet
nyc2=int((l2-t2)/dyc2)      # Number of y elements in outlet
nblockin=int(2*s2/delb_precise)+1
ncurvext=int(radius_ex*(np.pi*0.5-alpha_rot*np.pi/180)/delb_precise)+1

def equations(vars):
    x, y = vars
    eq1 = x**y - x_N/x_0
    eq2 = (1-x**(y+1))/(1-x) - s1/x_0
    return [eq1, eq2]

initial_guess = [1.1, 50]
solution = fsolve(equations, initial_guess)
nsl1=int(solution[1])+3
nsl2=nsl1
prog=1/solution[0]
# Modifiy the file name here
file_name='channel.geo'
last_point_id=0
last_line_id=0
last_transline_id=0
last_line_loop=0
last_surface_id=0
last_transurface=0
last_phys_surf=0
field_id=0


with open(file_name, 'w') as file:
    print(f"//Channel MESH FILE\n", file=file)

if ncurvext<3:
    ncurvext=3

Sum_geom=(1-(1/prog)**(nsl1-1))/(1-(1/prog))
x_0_real=s1/Sum_geom
x_N_temp=0
i=0
while x_N_temp<x_N2:
    i=i+1
    x_N_temp=x_0_real*((1/prog)**i)
nsl_2=i+1
#nsl_2=np.emath.logn((1/prog),1-(thick2/x_0_real)*(1-1/prog))+1
thick2=x_0_real*(1-(1/prog)**(nsl_2-1))/(1-(1/prog))
thick1=thick2

# Point definition

# Inlet points
point(0,0,0,h,1)
point(lc,0,0,h,2)
point(lc,s1,0,h,3)
point(0,s1,0,h,4)
point(0,d1-s2,0,h,5)
point(lc,d1-s2,0,h,6)
point(lc,d1,0,h,7)
point(0,d1,0,h,8)

# Line definition
line(1,2,1)
line(2,3,2)
line(3,4,3)
line(4,1,4)
line(3,6,5)
line(4,5,6)
line(5,6,7)
line(6,7,8)
line(7,8,9)
line(8,5,10)

#----------------Mesh definition algorithm--------------------------------------
transline(nxc1, 1, 1)
transline(nsl1, 1/prog, 2)
transline(nxc1, 1, 3)
transline(nsl1, prog, 4)
transline(nyc1, 1, 5)
transline(nyc1, 1, 6)
transline(nxc1, 1, 7)
transline(nsl2, prog, 8)
transline(nxc1, 1, 9)
transline(nsl2, 1/prog, 10)


loop((1,2,3,4),3)
loop((5,-7,-6,-3),4)
loop((7,8,9,10),5)

surface(3,"3")
surface(4,"4")
surface(5,"5")


transurface((1,2,3,4),3)
transurface((3,4,5,6),4)
transurface((5,6,7,8),5)


Thickness=thick1*1.03
Ratio=1/prog
Sum_geom=(1-(1/prog)**(nsl1-1))/(1-(1/prog))
Size=s1/Sum_geom
field_id=field_id+1



lines_to_add_rr = [
    'Physical Surface("VOLUME") = {3,4,5};',
    'Physical Line("wall") = {1,9};',
    'Physical Line("inlet") = {4,6,10};',
    'Physical Line("outlet") = {2,5,8};\n'
]


line_string = "{" + ', '.join(map(str, np.arange(1, last_line_loop+1, 1))) + "};"
lines_to_add2 = [
    f"Plane Surface(1)={line_string}"
]
#raise SystemExit

write_custom(lines_to_add_rr)

# Mesh generation and name
D = str(d1)
D = D.replace('.','_')
re = str(round(radius_ex,3))
re = re.replace('.','_')
ri = str(round(radius_in,3))
ri = ri.replace('.','_')
li = str(l1)
li = li.replace('.','_')
lo = str(l2)
lo = lo.replace('.','_')
mesh_suffix = 'D'+D+'_re'+re+'_ri'+ri+'_li'+li+'_lo'+lo
mesh_name = '"ch'+mesh_suffix+'.su2"'
meshing = [
    'Mesh 2;',
    'Save '+mesh_name+';\n'
]
write_custom(meshing)